In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import scipy.stats as stats
from scipy.stats import norm

import os
import time
import random
import math
import re
import subprocess
import sys
import warnings

import requests
import openpyxl
from openpyxl.styles import Alignment
from itertools import combinations, groupby, product
from dateutil.relativedelta import relativedelta
from datetime import date, datetime
import plotly.graph_objects as go
import plotly.io as pio


from IPython.core.display import display, HTML

import corr_trading as corr_trading

# Ignore all warnings
warnings.filterwarnings("ignore")


In [2]:
start_time = datetime.now()

In [3]:
asset_classes =["equity", 'crypto'] # See asset classes in the Extract Fin data repository for possible values

# Location
general_path = "/Users/arenquiestas/Documents/Projects/"
project_path = f"{general_path}Correlation Pairs Trading"
## Financial Data Database
sql_file_path = '/Users/arenquiestas/Documents/Projects/Financial Data Extract/Output/FINANCIAL DATA.db'

years_active = 10  # Number of years
years_analysis = 10
# years_train = 10
top_traded = 4000
max_price = 10000 # applies to current price, and when taking positions
benchmark_ticker = '^GSPC'
risk_free_prox = 'US 10-Year Treasury Yield'
# Parameters:
correlation_method = "pearson"

# recalculate_corr_pairs = "Y"
recalculate_engle_granger = "NEW"  # [NEW,ALL,NO]
engle_granger_sig_level = 0.01
recalculate_johansen = "NEW"  # [NEW,ALL,NO]
johansen_sig_level = 0.01
johansen_det_order = 0
johansen_k_ar_diff = 1
update_pairing = "N"

cl_sp = 0.95 # Confidence level for Spread Deviation
cl_st = 0.75 # Confidence level for Price Deviation
maintain_threshold = 0.05 # Hold the position until the price is within cl_st - maintian within the mean. Default is 0.05

# Pre-Process Data

In [4]:
all_tickers = corr_trading.load_sql_table('TICKERS_INFO',sql_file_path) 
dropped_tickers = corr_trading.load_sql_table('TICKERS_DROPPED',sql_file_path) 

In [5]:
# Load Index Data for Benchmarking

# Find the latest CSV file and read it into a DataFrame
index = corr_trading.load_sql_table('INDICES',sql_file_path) 
index = index[['Date', 'Ticker', 'Adj Close']]

# Filter, rename, set index, and calculate returns
benchmark = (index[index['Ticker'] == benchmark_ticker]
             .rename(columns={'Adj Close': 'market'})
             .set_index('Date')
             .loc[:, ['market']])

# Calculate the returns
benchmark['market_return_usd'] = benchmark['market'].diff()
benchmark['market_return_pct'] = benchmark['market'].pct_change()

# Add the MultiIndex
benchmark = benchmark.pipe(lambda df: df.set_axis(pd.MultiIndex.from_product([['Benchmark'], benchmark.columns]), axis=1))

fred = corr_trading.load_sql_table('FRED',sql_file_path)
fred = fred[['Date',risk_free_prox]]
fred[risk_free_prox] = fred[risk_free_prox]/100 # convert to decimal
fred[risk_free_prox] =(1 + fred[risk_free_prox] / 100) ** (1/252) - 1  # Assuming 252 trading days in a year
print('Risk Free Rate converted to daily')
risk_free = fred.set_index('Date').rename(columns={risk_free_prox:'risk_free_rate'})
risk_free = risk_free.pipe(lambda df: df.set_axis(pd.MultiIndex.from_product([['Benchmark'], risk_free.columns]), axis=1))
# risk_free['risk_free_rate'] = risk_free['risk_free_rate']/100


benchmark = pd.merge(left = risk_free, 
                     right = benchmark,
                     left_index=True,
                     right_index=True,
                     how = 'right')

Risk Free Rate converted to daily


In [6]:
fin_data = corr_trading.read_fin_data(asset_classes,sql_file_path)

equity asset class read
crypto asset class read


In [7]:
fin_data = corr_trading.remove_dropped(df=fin_data, dropped_df=dropped_tickers)

# Filter securities to include only those that have been active for at least the past 'y' years.
fin_data = corr_trading.filter_years(
    df=fin_data,
    all_tickers_df=all_tickers,
    years_active=years_active,
    start_time=start_time)

Dropped tickers removed
Filtered to years_active > 10


5820   2014-05-06 13:30:00
Name: firstTradeDateEpochUtc, dtype: datetime64[ns]

In [9]:
fin_data = fin_data.set_index(["Date"])

# Optional. Filter to Tickers wherein the latest price is < $10000.
filtered_df = fin_data[fin_data.index == fin_data.index[-1]]
filtered_df = filtered_df[filtered_df['Adj Close'] <= max_price]
fin_data = fin_data[fin_data['Ticker'].isin(filtered_df['Ticker'])]
fin_data = fin_data.reset_index()
print(f'Tickers with latest price above ${max_price} removed')

Tickers with latest price above $10000 removed


In [10]:
# Filter equity stocks to retain only the top 'n' most-traded stocks. This filtering applies exclusively to equity stocks.
fin_data = corr_trading.filter_most_traded(df=fin_data, top_n=top_traded)


# Pivot the data to have tickers as columns and 'Adj Close' as values
fin_data = fin_data.set_index("Date")[["Adj Close", "Ticker"]]
fin_data2 = fin_data.pivot(columns="Ticker", values="Adj Close")


# Identify tickers where the last row contains NaN values and drop those tickers
nan_columns = fin_data2.columns[fin_data2.iloc[-1].isna()]
fin_data2.drop(columns=nan_columns, inplace=True)


# Filter financial data to last 'years_analysis' years
fin_data2 = fin_data2.tail(252 * years_analysis)

# Handle NaN values in DataFrame by interpolation
fin_data2 = corr_trading.handle_nan(fin_data2)

## Remove Tickers with NaN. This is necessary because interpolation will only handle NaN in between 2 non-NaN cells.
### Some tickers may have gone public but prices are not available in YahooFinance until after some time. Example: all_tickers[all_tickers['Ticker'] == 'BIVI']["firstTradeDateEpochUtc"]
fin_data2.dropna(axis=1, how="any", inplace=True)
print('Tickers with NaN after all of the filters above have been removed')


Equity stocks filtered to Top 4000 most traded
Missing values interpolated.
Tickers with NaN after all of the filters above have been removed


In [11]:
fin_data3 = fin_data2.copy()

# # Drop columns where all values are the same
# fin_data3 = fin_data3.loc[:, (fin_data3 != fin_data3.iloc[0]).any()]

ticker_list = fin_data3.columns.tolist()

# Pairing

## Correlation Matrix

**Pearson**: Measures the linear relationship between two continuous variables. Assumes a normal distribution and calculates the strength and direction of the linear relationship.


In [12]:
# Correlation Matrix
corr_matrix = fin_data3.corr(method=correlation_method)

## Cointegration


### Johansen Test

- **Cointegration Test: Johansen Test**
  - The Johansen test evaluates whether multiple time series variables are cointegrated, indicating they share a stable long-term relationship despite short-term fluctuations. Unlike the Engle-Granger test, which examines pairwise relationships, Johansen's test handles multiple variables simultaneously to identify the number of cointegrating relationships and their associated vectors.
- **Maximum Eigenvalue Statistic**

  - In Johansen's test, the maximum eigenvalue statistic is one of the test statistics used to determine the presence of cointegration. It evaluates whether there is at least one cointegrating vector among the variables. The statistic is based on the largest eigenvalue of the system and provides insights into the strength of the most significant cointegrating relationship.

- **Trace Statistic**

  - The trace statistic in Johansen's test aggregates the eigenvalues across the system to assess the overall number of cointegrating vectors. It tests the null hypothesis that the number of cointegrating vectors is less than or equal to a specified number (typically the number of variables minus one). A significant trace statistic suggests the presence of cointegration.

- **Critical Values**

  - Critical values are thresholds determined from statistical tables or simulations that define the rejection regions for the test statistics (maximum eigenvalue and trace statistic). They indicate the cutoff points beyond which the test statistics' values are considered significant at a chosen significance level (e.g., 5% or 1%). Critical values vary based on the number of variables and the chosen significance level, and they are essential for interpreting the results of Johansen's test.

- **Strength of Relationship**

  - While cointegration itself doesn't quantify the strength of the relationship between variables, the test statistics and scores from Johansen's test provide insights into the magnitude of cointegration. Higher scores or more significant critical values indicate a stronger long-term relationship among the variables.

- **Direction of Relationship**
  - Johansen's test, like other cointegration tests, does not inherently determine the directionality (positive or negative) of the relationships. The direction is often interpreted based on economic theory or context specific to the variables being analyzed.


In [13]:
johansen_results = corr_trading.calculate_johansen(
    recalculate_johansen,
    ticker_list,
    fin_data3,
    project_path,
    det_order=0,
    k_ar_diff=johansen_k_ar_diff,
    johansen_sig_level=johansen_sig_level,
)

0 new combinations to be calculated
Cointegration results exist for all combinations


## Final Pairing


In [14]:
# Pair stocks based on correlation, selecting the highest correlated pair that is cointegrated in both Engle-Granger and Johansen tests.

if update_pairing == 'Y':
    
    pairs_0 = corr_trading.pairing(
        corr_matrix,
        johansen_results)

    pairs_0 = pairs_0.sort_values(by="correlation", ascending=False).reset_index(drop=True)
    pairs_0.to_csv(f'/Users/arenquiestas/Documents/Projects/Correlation Pairs Trading/Outputs/2_Results/Pairing/PAIRS {datetime.now().strftime("%Y-%m-%d")}.csv', index=False)

elif update_pairing != 'Y':
    
    pairs_0_file = sorted([file for file in os.listdir(f'/Users/arenquiestas/Documents/Projects/Correlation Pairs Trading/Outputs/2_Results/Pairing') if file.endswith('.csv') and file.startswith('PAIRS')])[-1]
    pairs_0 = pd.read_csv(f"/Users/arenquiestas/Documents/Projects/Correlation Pairs Trading/Outputs/2_Results/Pairing/{pairs_0_file}")

pairs_list = pairs_0[["stock1", "stock2"]].values.tolist()

In [15]:
pairs_1 = pd.DataFrame()
pairs_1.index = fin_data3.index.copy()
pairs_1.columns = pd.MultiIndex.from_tuples([], names=["", ""])

# Create a new DataFrame to store spreads and prices, with paired tickers indicated in the level 0 column
for stock1, stock2 in zip(pairs_0["stock1"], pairs_0["stock2"]):
    temp = fin_data3[[stock1, stock2]]
    temp["spread"] = temp[stock1] - temp[stock2]
    temp.columns = pd.MultiIndex.from_product([[f"{stock1}_{stock2}"], temp.columns])
    pairs_1 = pd.merge(
        left=pairs_1, right=temp, right_index=True, left_index=True, how="outer"
    )

# Positions

## Deviation Threshold: Z-Score


In [20]:
pairs_z_score = corr_trading.dev_thresh_z_score(
    pairs_list=pairs_list,
    pairs_df=pairs_1,
    cl_spread= cl_sp,
    cl_stock= cl_st)

# Append Risk Free Rate and Market Returns
pairs_z_score = pd.merge(left = pairs_z_score,
                            right = benchmark,
                            left_index=True,
                            right_index=True,
                            how = 'left')
            


Total Absolute Return (Z-Score Spread: 0.95, Stock: 0.75,  Maintain: 0.05): 174.38404393196106


# Metrics

In [21]:
# Prepare the DataFrame by selecting relevant columns and filling missing total returns with market returns

metrics = pd.DataFrame()

for pair in [a for a in pairs_z_score.columns.get_level_values(0).unique().to_list() if a != 'Benchmark']:
    
    # Calculate the excess return as the difference between total return and risk-free rate
    pairs_z_score[(pair,'tot_return_pct')] = pairs_z_score[(pair,'tot_return_pct')].fillna(pairs_z_score[('Benchmark','market_return_pct')])
    pairs_z_score[(pair,'excess_return_pct')] = pairs_z_score[(pair,'tot_return_pct')] - pairs_z_score[('Benchmark','risk_free_rate')]

    df = pairs_z_score[[pair, 'Benchmark']].droplevel(level=0, axis=1)
    
    df.reset_index(inplace=True)

    # Calculate the average excess return
    average_excess_return_pct = df['excess_return_pct'].mean()

    # Calculate downside deviation for negative excess returns
    downside_returns = df['tot_return_pct'][df['tot_return_pct'] < 0]
    downside_deviation = np.std(downside_returns) if len(downside_returns) > 0 else 0

    # Calculate the standard deviation of all total returns for Sharpe Ratio
    total_deviation = np.std(df['tot_return_pct'])

    # Calculate Beta using linear regression (slope of the regression line)
    X = sm.add_constant(df['market_return_pct'])  # Add a constant term for the intercept
    model = sm.OLS(df['tot_return_pct'], X).fit()
    beta = model.params[1]  # The slope coefficient represents Beta

    # Calculate expected return using the CAPM formula
    expected_return = df['risk_free_rate'].mean() + beta * (df['market_return_pct'].mean() - df['risk_free_rate'].mean())

    # Calculate Jensen's Alpha
    jensens_alpha = average_excess_return_pct - (expected_return - df['risk_free_rate'].mean())

    # Calculate the Sortino Ratio
    sortino_ratio = average_excess_return_pct / downside_deviation if downside_deviation != 0 else np.nan

    # Calculate the Sharpe Ratio
    sharpe_ratio = average_excess_return_pct / total_deviation if total_deviation != 0 else np.nan

    # Calculate the Treynor Ratio
    treynor_ratio = average_excess_return_pct / beta if beta != 0 else np.nan

    # Calculate Tracking Error (standard deviation of the difference between portfolio and market returns)
    tracking_error = np.std(df['tot_return_pct'] - df['market_return_pct'])

    # Calculate Information Ratio
    information_ratio = average_excess_return_pct / tracking_error if tracking_error != 0 else np.nan

    # Trade Metrics

    # Calculate the best and worst trades in USD and percentage
    best_trade_usd = df["tot_return_usd"].max()
    best_trade_pct = df["tot_return_pct"].max()
    worst_trade_usd = df["tot_return_usd"].min()
    worst_trade_pct = df["tot_return_pct"].min()

    # Calculate average win and loss metrics
    average_win_usd = df.loc[df["tot_return_usd"] > 0, "tot_return_usd"].mean()
    average_win_pct = df.loc[df["tot_return_pct"] > 0, "tot_return_pct"].mean()
    average_loss_usd = df.loc[df["tot_return_usd"] < 0, "tot_return_usd"].mean()
    average_loss_pct = df.loc[df["tot_return_pct"] < 0, "tot_return_pct"].mean()

    # Calculate average trade return
    average_trade_return_usd = df["tot_return_usd"].mean()
    average_trade_return_pct = df["tot_return_pct"].mean()

    # Calculate total trade return
    total_trade_return = df["tot_return_usd"].sum()

    # Calculate trade winning percentage
    try:
        trade_winning_percentage = len(df[df["tot_return_usd"] > 0]) / len(df[df[f'position_{pair.split("_")[0]}'] != ""])
    except ZeroDivisionError:
        trade_winning_percentage = 0  # Handle case where there are no trades

    num_trade = len(df[df[f'position_{pair.split("_")[0]}'] != ''])
    df2 = df[['Date',f'position_{pair.split("_")[0]}']]
    df2 = df2.rename(columns={f'position_{pair.split("_")[0]}':'Position'})
    average_days_live = corr_trading.calculate_average_days_live(df2)

    # Extract month and year from the Date column for aggregation
    df['Month'] = df['Date'].dt.to_period('M')
    df['Year'] = df['Date'].dt.to_period('Y')
    df['Return'] = df['tot_return_usd']

    # Aggregate returns by month
    monthly_returns = df.groupby('Month')['Return'].sum()

    # Aggregate returns by year
    yearly_returns = df.groupby('Year')['Return'].sum()

    # Calculate monthly metrics
    num_months = len(monthly_returns)
    num_winning_months = len(monthly_returns[monthly_returns > 0])
    num_losing_months = len(monthly_returns[monthly_returns < 0])

    # Calculate winning months percentage
    winning_months_percentage = (num_winning_months / num_months) * 100 if num_months > 0 else 0

    # Calculate average returns for winning and losing months
    avg_return_winning_month = monthly_returns[monthly_returns > 0].mean() if num_winning_months > 0 else 0
    avg_return_losing_month = monthly_returns[monthly_returns < 0].mean() if num_losing_months > 0 else 0

    # Get best and worst month returns
    best_month_return = monthly_returns.max()
    worst_month_return = monthly_returns.min()

    # Calculate yearly metrics
    num_years = len(yearly_returns)
    num_winning_years = len(yearly_returns[yearly_returns > 0])
    num_losing_years = len(yearly_returns[yearly_returns < 0])

    # Calculate winning years percentage
    winning_years_percentage = (num_winning_years / num_years) * 100 if num_years > 0 else 0

    # Calculate average returns for winning and losing years
    avg_return_winning_year = yearly_returns[yearly_returns > 0].mean() if num_winning_years > 0 else 0
    avg_return_losing_year = yearly_returns[yearly_returns < 0].mean() if num_losing_years > 0 else 0

    # Get best and worst year returns
    best_year_return = yearly_returns.max()
    worst_year_return = yearly_returns.min()

    # Store results in a DataFrame
    curve_metrics = pd.DataFrame({
        'Metric': [
            'Sortino Ratio', 
            'Sharpe Ratio', 
            'Treynor Ratio', 
            'Jensen\'s Alpha', 
            'Beta', 
            'Tracking Error', 
            'Information Ratio',
        ],
        'Value': [
            sortino_ratio, 
            sharpe_ratio, 
            treynor_ratio, 
            jensens_alpha, 
            beta, 
            tracking_error, 
            information_ratio,
        ]
    })

    # Store results in a DataFrame
    trade_metrics = pd.DataFrame({
        'Metric': [
            "Best Trade ($)",
            "Best Trade (%)",
            "Worst Trade ($)",
            "Worst Trade (%)",
            "Average Win ($)",
            "Average Win (%)",
            "Average Loss ($)",
            "Average Loss (%)",
            "Average Trade Return ($)",
            "Average Trade Return (%)",
            'Winning Trade (%)',
            'Total Return ($)',
            'Number of Trades',
            'Avg. Days in Trade'
        ],
        'Value': [
            best_trade_usd,
            best_trade_pct,
            worst_trade_usd,
            worst_trade_pct,
            average_win_usd,
            average_win_pct,
            average_loss_usd,
            average_loss_pct,
            average_trade_return_usd,
            average_trade_return_pct,
            trade_winning_percentage,
            total_trade_return,
            num_trade,
            average_days_live
        ]
    })

    # Store results in a DataFrame
    time_metrics = pd.DataFrame({
        'Metric': [
            "Winning Years (%)",
            "Average Return for Winning Year (%)",
            "Average Return for Losing Year (%)",
            "Best Year Return (%)",
            "Worst Year Return (%)",
            "Winning Months (%)",
            "Average Return for Winning Month (%)",
            "Average Return for Losing Month (%)",
            "Best Month (% Return)",
            "Worst Month (% Return)",
        ],
        'Value': [
            winning_years_percentage,
            avg_return_winning_year,
            avg_return_losing_year,
            best_year_return,
            worst_year_return,
            winning_months_percentage,
            avg_return_winning_month,
            avg_return_losing_month,
            best_month_return,
            worst_month_return
        ]
    })

    # Final DF
    for x,y in zip(['curve_metrics','trade_metrics','time_metrics'], ['CURVE','TRADE','TIME']):
        globals()[x] = globals()[x].set_index('Metric').T
        new_columns = pd.MultiIndex.from_product([[y], globals()[x].columns], names=[None,None])
        globals()[x].columns = new_columns
        
    temp = pd.concat([curve_metrics,
                            trade_metrics,
                            time_metrics], axis =1)
    temp['PAIR'] = pair
    temp = temp.set_index('PAIR')
    
    metrics = pd.concat([metrics,temp])
    
metrics.to_excel(f'{project_path}/Outputs/2_Results/Trade Metrics/Metrics.xlsx', sheet_name='METRICS')

# Daily Trade Signals

In [23]:
curr_day_positions = []
curr_day_index = pairs_z_score.index[-1]

# list_of_cols
temp_list = [[level_0] + [col for col in pairs_z_score[level_0].columns if 'position' in col.lower()]
               for level_0 in pairs_z_score.columns.levels[0]]
temp_list = [item for item in temp_list if len(item) > 1]


# Identify the level 1 columns containing 'position'
for pairs in temp_list:
    temp_df = pairs_z_score[pairs[0]]
    if temp_df.iloc[-1][pairs[1]] and temp_df.iloc[-1][pairs[2]]:
            curr_day_positions.append(pairs[0])
    else:
        pass

# Create a new df where positions are to be taken
todays_trade_signals = pairs_z_score.loc[:, pairs_z_score.columns.get_level_values(0).isin(curr_day_positions + ['Benchmark'])]

if todays_trade_signals.empty:
        print('No trade signals today')
else:
        print(f'Trade signals for: {curr_day_positions}')

Trade signals for: ['HRTG_ITP']


# Visualisation

In [96]:
fig_list = []
curve_table_list = []
trade_table_list = []
time_table_list = []

for pair in curr_day_positions:
    temp = todays_trade_signals[[pair,'Benchmark']]
    temp = temp.droplevel(level=0, axis=1)
    temp = temp[[col for col in todays_trade_signals[['HRTG_ITP','Benchmark']].droplevel(level=0, axis=1).columns.tolist() 
                    if col.endswith('pct')
                    or col == 'risk_free_rate']]
    
    temp['return_cumulative_market'] = (1 + temp['market_return_pct']).cumprod() - 1
    temp['return_cumulative_pair'] = (1 + temp['tot_return_pct']).cumprod() - 1
    
    globals()[f'fig_{pair.lower()}'] = go.Figure()
    
    # Add first time series
    globals()[f'fig_{pair.lower()}'].add_trace(go.Scatter(x=temp.index, y=temp['return_cumulative_market'], mode='lines', name='Cumulative Market Return', line=dict(color='blue')))

    # Add second time series
    globals()[f'fig_{pair.lower()}'].add_trace(go.Scatter(x=temp.index, y=temp['return_cumulative_pair'], mode='lines', name='Cumulative Pairs_Strategy Return', line=dict(color='green')))

        # Update layout
    globals()[f'fig_{pair.lower()}'].update_layout(
        title='Cumulative Market Returns',
        xaxis_title='Date',
        yaxis_tickformat='.1%',
        yaxis_title='Cumulative Return (%)',
        legend_title='Legend',
        template='plotly_white'
    )
    
    globals()[f'fig_html_{pair.lower()}'] = pio.to_html(globals()[f'fig_{pair.lower()}'], full_html=False)
    
    fig_list.append(f'fig_html_{pair.lower()}')
    
    # HTML Tables
    
    ### Curve Metrics
    temp = metrics.loc[[pair]]
    temp = temp['CURVE'].T
    temp.columns.name = None
    
    html_table = temp.to_html()
    headline = "<h2>CURVE</h2>"
    globals()[f'curve_metrics_html_{pair.lower()}'] = headline + html_table

    globals()[f'curve_metrics_html_{pair.lower()}'] = globals()[f'curve_metrics_html_{pair.lower()}'].replace('<thead>', '<thead style="background-color: white; color: black;">')
    # Change body fill color to white and text color to black
    globals()[f'curve_metrics_html_{pair.lower()}'] = globals()[f'curve_metrics_html_{pair.lower()}'].replace('<tbody>', '<tbody style="background-color: white; color: black;">')

    curve_table_list.append(f'curve_metrics_html_{pair.lower()}')

    ### Trade Metrics
    temp = metrics.loc[[pair]]
    temp = temp['TRADE'].T
    temp.columns.name = None
    
    html_table = temp.to_html()
    headline = "<h2>TRADE</h2>"
    globals()[f'trade_metrics_html_{pair.lower()}'] = headline + html_table

    globals()[f'trade_metrics_html_{pair.lower()}'] = globals()[f'trade_metrics_html_{pair.lower()}'].replace('<thead>', '<thead style="background-color: white; color: black;">')
    # Change body fill color to white and text color to black
    globals()[f'trade_metrics_html_{pair.lower()}'] = globals()[f'trade_metrics_html_{pair.lower()}'].replace('<tbody>', '<tbody style="background-color: white; color: black;">')

    trade_table_list.append(f'trade_metrics_html_{pair.lower()}')
    
    ### Time Metrics
    temp = metrics.loc[[pair]]
    temp = temp['TIME'].T
    temp.columns.name = None
    
    html_table = temp.to_html()
    headline = "<h2>TIME</h2>"
    globals()[f'time_metrics_html_{pair.lower()}'] = headline + html_table

    globals()[f'time_metrics_html_{pair.lower()}'] = globals()[f'time_metrics_html_{pair.lower()}'].replace('<thead>', '<thead style="background-color: white; color: black;">')
    # Change body fill color to white and text color to black
    globals()[f'time_metrics_html_{pair.lower()}'] = globals()[f'time_metrics_html_{pair.lower()}'].replace('<tbody>', '<tbody style="background-color: white; color: black;">')

    time_table_list.append(f'time_metrics_html_{pair.lower()}')

In [108]:
# Combine all tables

signals_path = f'{project_path}/Outputs/3_Signals/RUN_{start_time.strftime("%Y%m%d")}'

if not os.path.exists(signals_path):
    os.makedirs(signals_path)
    print(f'{signals_path} Created')


for fig, c_table, tr_table, ti_table, pair in zip(fig_list, curve_table_list, trade_table_list, time_table_list, curr_day_positions):
    html_content = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Plotly Graph with Tables</title>
            <style>
                /* Add some basic styling */
                body {{
                    font-family: Arial, sans-serif;
                    margin: 20px;
                }}

                table {{
                    width: 100%;
                    border-collapse: collapse;
                    margin-top: 20px;
                }}

                th, td {{
                    padding: 10px;
                    border: 1px solid #ddd;
                    text-align: left;
                }}

                th {{
                    background-color: #f4f4f4;
                }}
            </style>
        </head>
        <body>

            <h1>Plotly Graph with Tables</h1>

            <!-- Plotly Graph -->
            <div id="plotly_graph">
                {globals()[fig]}
            </div>

            <!-- HTML Table 1 -->
            {globals()[c_table]}

            <!-- HTML Table 2 -->
            {globals()[tr_table]}
            
            <!-- HTML Table 3 -->
            {globals()[ti_table]}

            <!-- Include the Plotly.js library -->
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>

        </body>
        </html>
        """

        # Save the combined HTML to a file
    with open(f'{signals_path}/{pair}-{start_time.strftime("%Y-%m-%d")}.html', 'w') as file:
        file.write(html_content)
    
    print(f'Dashboard for {pair} saved in {signals_path}')

Dashboard for HRTG_ITP saved in /Users/arenquiestas/Documents/Projects/Correlation Pairs Trading/Outputs/3_Signals/RUN_20240826


# Notify

In [115]:
def send_imessage(phone_number, message):
    """
    Send an iMessage to a specified phone number.
    
    :param phone_number: The recipient's phone number.
    :param message: The message to send.
    """
    apple_script = f'''
    tell application "Messages"
        set targetService to 1st service whose service type = iMessage
        set targetBuddy to buddy "{phone_number}" of targetService
        send "{message}" to targetBuddy
    end tell
    '''
    
    subprocess.run(['osascript', '-e', apple_script])

if __name__ == "__main__":
    phone_number = "+447588448586"  # Replace with your phone number
    message = f'''Correlation Pairs Trading
        Start: {start_time.strftime("%Y-%m-%d  %H:%M")}
        End: {datetime.now().strftime("%Y-%m-%d  %H:%M")}
        Trade Signals for: {curr_day_positions}
            '''
    
    send_imessage(phone_number, message)


notified
